In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error

# --------------------------
# 1. Daily Input Collection
# --------------------------
df = pd.read_csv("Mental Health Dataset.csv")
df.drop(columns=["Timestamp"], inplace=True)
df.fillna("Unknown", inplace=True)
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace("\r", "").str.replace("\n", "")

# --------------------------
# 2. Stress Analysis
# --------------------------
cat_cols = ["Gender", "Country", "Occupation", "self_employed", "family_history", "treatment",
            "Growing_Stress", "Changes_Habits", "Mental_Health_History", "Mood_Swings",
            "Coping_Struggles", "Work_Interest", "Social_Weakness", "mental_health_interview", "care_options"]

encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# Convert Days_Indoors to numeric
df["Days_Indoors"] = pd.to_numeric(df["Days_Indoors"], errors='coerce').fillna(0)

# Stress score calculation
stress_score = df["Growing_Stress"] + df["Changes_Habits"] + df["Coping_Struggles"]
conditions = [
    (stress_score <= 1),
    (stress_score == 2),
    (stress_score >= 3)
]
stress_labels = [0, 1, 2]  # 0 = Low, 1 = Medium, 2 = High
df["Stress_Level"] = np.select(conditions, stress_labels)

# --------------------------
# 3. Risk Assessment (Mood Score)
# --------------------------
df["Mood_Score"] = 10 - (df["Mood_Swings"] + df["Social_Weakness"] + (1 - df["Work_Interest"])) * 3.33
df["Mood_Score"] = df["Mood_Score"].clip(0, 10)

# Feature scaling
features = df.drop(columns=["Stress_Level", "Mood_Score"])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# --------------------------
# 4. Model Training
# --------------------------
# Classification: Stress Level
X_cls = X_scaled
y_cls = df["Stress_Level"]
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)
cls_model = RandomForestClassifier(n_estimators=100, random_state=42)
cls_model.fit(X_train_c, y_train_c)
cls_preds = cls_model.predict(X_test_c)
print("\nStress Level Classification Report:")
print(classification_report(y_test_c, cls_preds))

# Regression: Mood Score
X_reg = X_scaled
y_reg = df["Mood_Score"]
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
reg_model = RandomForestRegressor(n_estimators=100, random_state=42)
reg_model.fit(X_train_r, y_train_r)
reg_preds = reg_model.predict(X_test_r)
print("\nMood Score Regression RMSE:", np.sqrt(mean_squared_error(y_test_r, reg_preds)))

# --------------------------
# 5. Preventive Interventions
# --------------------------
stress_pred = cls_model.predict(X_scaled)
mood_pred = reg_model.predict(X_scaled)

# Copy original data with predictions
df_output = df.copy()
df_output["Predicted_Stress_Level"] = stress_pred
df_output["Predicted_Mood_Score"] = mood_pred.round(2)

# Generate recommendations based on predictions
def get_recommendation(stress, mood):
    if stress == 2 and mood < 4:
        return "Seek professional help"
    elif stress == 1 and mood < 6:
        return "Try meditation & reduce workload"
    elif stress == 0 and mood > 7:
        return "Maintain current routine"
    else:
        return "Regular exercise & social interaction"

df_output["Recommendation"] = df_output.apply(lambda row: get_recommendation(row["Predicted_Stress_Level"], row["Predicted_Mood_Score"]), axis=1)

# Save final output
df_output.to_csv("mental_health_predictions.csv", index=False)
print("\nPredictions saved to 'mental_health_predictions.csv'")



Stress Level Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11309
           1       1.00      1.00      1.00     18513
           2       1.00      1.00      1.00     28651

    accuracy                           1.00     58473
   macro avg       1.00      1.00      1.00     58473
weighted avg       1.00      1.00      1.00     58473


Mood Score Regression RMSE: 1.7713074022713274e-13

Predictions saved to 'mental_health_predictions.csv'
